# Solving continuous time economic models: Poisson income process

This Jupyter notebook presents Julia code that solves a continuous time version of a heterogeneous agents economy. Specifically, it solves a version of the Huggett model in which agents solve a consumption-savings problem subject to a borrowing constraint and idiosyncratic shocks. This version of the model assumes that the income process for agents follows a Markov process with a Poisson probability of switching between two possible income states.

The model and it's solution method is described in detail in the accompanying PDF document. Note, too, that an additional Jupyter notebook solves the same model but with a diffusion process governing the income state variable. 

#### Defining a composite type for the model object

The first thing we do before anything else is declare a composite type, which contains (most of) the fields that will be used in the solution to the model. Each model parameter, variable, and data storage matrix is given a particular type, which helps increase the speed with which the code can be run. 

In [1]:
type Huggett_model
  sigma::Float64            # risk aversion coefficient
  rho::Float64              # discount rate.
  z1::Float64               # Income in state 1
  z2::Float64               # Income in state 2
  z::Array{Float64}         # Vector for the two incomes
  la1::Float64              # State 1 switching intensity
  la2::Float64              # State 2 switching intensity
  la::Array{Float64}        # Vector for the two intensities
  I::Int64                  # Number of grids for assets
  Ir::Int64                 # Number of interest rate iterations
  amin::Float64             # Min assets = borrowing constraint
  amax::Float64             # Max assets = upper bound of state space
  rmin::Float64             # Min interest rate
  rmax::Float64             # Max interest rate
  agrid::Vector{Float64}    # Grid space for assets
  aa::Array{Float64}        # Matrix for asset grid space
  zz::Array{Float64}        # Matrix for income grid space
  da::Float64               # This is the implied grid step size
  maxit::Int64              # Maximum iterations for HJB solution algorithm
  crit::Float64             # Tolerance for HJB equation solution   
  crit_S::Float64           # Tolerance for asset mkt clearing
  Delta::Float64            # Time step-size for use in approximating derivatives
  Aswitch::SparseMatrixCSC{Float64,Int64}    # Transition matrix
  v0::Array{Float64}        # Initial guess for value function
  r0::Float64               # Intial guess for interest rate
end

Next we create the model object itself. We do this by defining a function, which takes the deep parameters of the model as default arguments, and produces all of the other model objects that can be pre-computed. This includes the grid spaces for assets and income, the initial guess for the HJB value function, and the transition matrix used in computation of the HJB equation.

The final command in this cell produces an instance of the model object (`Huggett_model`), which we will use when calling any of the paramters, variables, data storage matrices, or pre-computed model objects. 

In [12]:
function Huggett_model(;sigma::Float64=1.5,
  rho::Float64=0.05,
  I::Int64=1000,
  Ir::Int64=40,
  amin::Float64=-0.15,
  amax::Float64=5.0,
  rmin::Float64=0.01,
  rmax::Float64=0.04,
  z1::Float64=0.1,
  z2::Float64=0.2,
  la1::Float64=1.2,
  la2::Float64=1.5,
  maxit::Int64=100,
  crit::Float64=1e-6,
  crit_S::Float64=1e-5,
  Delta::Float64=1000.0,
  r0::Float64=0.03)

  z       = hcat(z1,z2)
  la      = hcat(la1,la2)
  agrid   = collect(linspace(amin,amax,I))
  aa      = hcat(agrid, agrid)
  zz      = ones(I,1)*z
  da      = (amax-amin)/(I-1)
  v0 = hcat( (z[1] + r0.*agrid).^(1-sigma)/(1-sigma)/rho, (z[2] + r0.*agrid).^(1-sigma)/(1-sigma)/rho )

  Aswitch =  vcat( hcat(-speye(I)*la[1], speye(I)*la[1]), hcat(speye(I)*la[2],-speye(I)*la[2]) )

  Huggett_model(sigma, rho, z1, z2, z, la1, la2, la, I, Ir, amin, amax, rmin, rmax, agrid, aa, zz, da, maxit, crit, crit_S, Delta, Aswitch,  v0, r0)

end

# Creates an instance of the model
mod = Huggett_model();

Next I define several useful functions for use in solving the model. The first three are, the CRRA utility function, the derivative of the utility function, and the inverse of the derivative of the utility function. The final function solves the Kolmogorov Forward Equation for the model using the transition matrix found by solving the HJB equation. 

In [3]:
function util(x; sigma::Float64=mod.sigma)
  u = x.^(1-sigma)/(1-sigma)
  return u
end

function utilprime(x; sigma::Float64=mod.sigma)
  uprime = x.^(-sigma)
  return uprime
end

function utilprimeinv(x; sigma::Float64=mod.sigma)
  uprimeinv = x.^(-1/sigma)
  return uprimeinv
end

# Function to compute the KFE
function KFE(A::SparseMatrixCSC{Float64,Int64}; I::Int64=mod.I, da::Float64=mod.da)
  AT          = A'
  b           = zeros(2*I,1)
  i_fix       = 1    # fix one value so that matrix is not singular
  b[i_fix]    = 0.1
  row         = hcat(zeros(1,i_fix-1), 1, zeros(1,2*I-i_fix))
  AT[i_fix,:] = row
  #Solve linear system
  gg          = AT\b          # A'*gg = b --> gg = (A')^(-1)*b
  g_sum       = gg'*ones(2*I,1)*da
  gg          = gg./g_sum     # Normalize so sum(gg)*da) = 1
  g           = hcat(gg[1:I], gg[I+1:2*I])
  #check1      = g[:,1]'*ones(mod.I,1)*mod.da
  #check2      = g[:,2]'*ones(mod.I,1)*mod.da
  return g
end

KFE (generic function with 1 method)

Next, we initialize several data storage arrays. This turns out to be important, as Julia does not store arrays/data that are initialized and produced inside loops. Thus, anything that we might want access to after convergence of the algorithm (e.g. the equilibrium interest rate, the value function, policy functions) needs to be initialized here. 

In [4]:
dVf         = zeros(mod.I,2)          # Forward difference in each state
dVb         = zeros(mod.I,2)          # Backward difference in each state
c           = zeros(mod.I,2)          # Consumption in each state
A           = sparse(zeros(2*mod.I,2*mod.I))   # Transition matrix
r_r         = zeros(mod.Ir,1)         # Interest rate guesses
rmin_r      = zeros(mod.Ir,1)         # Lower bound for bisection
rmax_r      = zeros(mod.Ir,1)         # Upper bound for bisection
g_r         = zeros(mod.I,2,mod.Ir)   # Stationary distribution
adot        = zeros(mod.I,2,mod.Ir)   # Policy function
V_r         = zeros(mod.I,2,mod.Ir)   # Value function
V           = zeros(mod.I,2)          # Value function
S           = zeros(mod.Ir,1)         # Aggregate asset supply
r           = mod.r0                  # Current interest rate guess
dist        = zeros(mod.maxit,1)      # Distance between value function guesses
r_end       = 0                       # Last iteration value

0

Now we get to the meat of the code. This code block runs the algorithm for solving the model and the associated equilibrium. 

#### Inner loop of the algorithm
After the outer loop has updated (initalized) the interest rate guess, the inner loop proceeds as follows:
* Construct the forward difference approximations of the derivatives of the value function
* Use the "upwind scheme" to compute consumption, utility, and the savings policy rules
* Use the savings policy rules to construct the transition matrix. This construction is carried out using sparse matrix methods to save memory (these methods are also fast). 
* Construct and solve the linear system of equations that constitute the HJB equation(s) across asset and income states.

#### Outer loop of the algorithm
After the inner loop has solved the HJB equation, the transition matrix computed on the final iteration is used to solve the KFE for the stationary distribution. The outer loop then uses this stationary distribution to compute the aggregate asset supply. We then update the interest rate via a bisection method until assets are in zero net supply. 



In [5]:
for ir in 1:mod.Ir
  r_r[ir]     = r
  rmin_r[ir]  = mod.rmin
  rmax_r[ir]  = mod.rmax

  if ir>1
    mod.v0 = V_r[:,:,ir-1]
  end

  v = mod.v0

  for n = 1:mod.maxit
    V               = v
    # forward difference
    dVf[1:mod.I-1,:]    = (V[2:mod.I,:]-V[1:mod.I-1,:])/mod.da
    dVf[mod.I,:]        = utilprime(mod.z + r.*mod.amax)
    # backward difference
    dVb[1,:]            = utilprime(mod.z + r.*mod.amin)  #state constraint boundary condition
    dVb[2:mod.I,:]      = (V[2:mod.I,:]-V[1:mod.I-1,:])/mod.da

    #consumption and savings with forward difference
    cf              = utilprimeinv(dVf)
    ssf             = mod.zz + r.*mod.aa - cf
    #consumption and savings with backward difference
    cb              = utilprimeinv(dVb)
    ssb             = mod.zz + r.*mod.aa - cb
    #consumption and derivative of value function at steady state
    c0              = mod.zz + r.*mod.aa
    dV0             =utilprime(c0)

    # dV_upwind makes a choice of forward or backward differences based on the sign of the drift
    If              = ssf .> 0    # positive drift --> forward difference
    Ib              = ssb .< 0    # negative drift --> backward difference
    I0              = (1-If-Ib)   # at steady state
    dV_Upwind       = dVf.*If + dVb.*Ib + dV0.*I0    #important to include third term
    c               = utilprimeinv(dV_Upwind)
    u               = util(c)

    # Construct transition matrix 
    X               = - min(ssb,0)/mod.da
    Y               = - max(ssf,0)/mod.da + min(ssb,0)/mod.da
    Z               = max(ssf,0)/mod.da
        
    A1              = spdiagm(Y[:,1],0,mod.I,mod.I) + spdiagm(X[2:mod.I,1],-1,mod.I,mod.I) + spdiagm(Z[1:mod.I-1,1],1,mod.I,mod.I)
    A2              = spdiagm(Y[:,2],0,mod.I,mod.I) + spdiagm(X[2:mod.I,2],-1,mod.I,mod.I) + spdiagm(Z[1:mod.I-1,2],1,mod.I,mod.I)
    A               = vcat(hcat(A1, sparse(zeros(mod.I,mod.I))), hcat(sparse(zeros(mod.I,mod.I)), A2)) + mod.Aswitch
    B               = (mod.rho + 1/mod.Delta)*speye(2*mod.I) - A

    u_stacked       = vcat(u[:,1], u[:,2])
    V_stacked       = vcat(V[:,1], V[:,2])

    # Set up and solve linear system of eqns    
    b               = u_stacked + V_stacked/mod.Delta
    V_stacked       = B\b             

    V               = hcat(V_stacked[1:mod.I], V_stacked[mod.I+1:2*mod.I])

    Vchange         = V - v
    v               = V

    dist[n] = maximum(maximum(abs(Vchange)))
    if dist[n]<mod.crit
      @printf("-----------------------\n")
      @printf("Interest rate = %f\n", r)
      @printf("Value Function Converged, Iteration = %i\n", n)
      break
    end

  end

  ## Solve the Kolmogorov Forward equations
  g = KFE(A)

  # Pack up output
  g_r[:,:,ir] = g
  adot[:,:,ir] = mod.zz + r.*mod.aa - c
  V_r[:,:,ir] = V
  S[ir] = (g[:,1]'*mod.agrid*mod.da + g[:,2]'*mod.agrid*mod.da)[1,1]

  # Check for asset market clearing, update the interest rate
  r_end = ir

  if S[ir]>mod.crit_S
    @printf("Excess Supply\n")
    mod.rmax = r
    r = 0.5*(r+mod.rmin)
  elseif S[ir]<-mod.crit_S
    @printf("Excess Demand\n")
    mod.rmin = r
    r = 0.5*(r+mod.rmax)
  elseif abs(S[ir])<mod.crit_S
    @printf("Equilibrium Found, Interest rate = %f/n", r)
    break
  end

end


-----------------------
Interest rate = 0.030000
Value Function Converged, Iteration = 8
Excess Demand
-----------------------
Interest rate = 0.035000
Value Function Converged, Iteration = 5
Excess Demand
-----------------------
Interest rate = 0.037500
Value Function Converged, Iteration = 5
Excess Demand
-----------------------
Interest rate = 0.038750
Value Function Converged, Iteration = 5
Excess Demand
-----------------------
Interest rate = 0.039375
Value Function Converged, Iteration = 5
Excess Demand
-----------------------
Interest rate = 0.039688
Value Function Converged, Iteration = 4
Excess Demand
-----------------------
Interest rate = 0.039844
Value Function Converged, Iteration = 4
Excess Demand
-----------------------
Interest rate = 0.039922
Value Function Converged, Iteration = 4
Excess Demand
-----------------------
Interest rate = 0.039961
Value Function Converged, Iteration = 4
Excess Demand
-----------------------
Interest rate = 0.039980
Value Function Converged

### Graphs

Finally, I use the Gadfly package to plot the savings policy function across the two income states, and to plot the distribution of assets across the two income states. Note that Gadfly does not seem to have an easily accessible legend function, and so let it be known that red lines are associated with the high income state, and blue lines are associated with the low income state. 

In [6]:
using Gadfly


INFO: Precompiling module Gadfly...


#### Savings policy function

Note that the savings policy function differs across income states. In the low income state (blue), agents are always dissaving, regardless of their current wealth/assets. Notice, too, that at the borrowing constraint, the derivative of the low income savings function is negative infinity: any additional wealth would be entirely dissaved away, taking the agent back to the borrowing constraint again. 

In the high income state (red), agents are saving over most of the asset state space. These agents will dissave when assets are very high. 

We can see that high income agents have a precautionary saving motive, and are self-insuring against the possibility of hitting the borrowing constraint if/when they enter the low income state. Low income agents always dissave in order to smooth consumption across income states. 

In [7]:
draw(SVG(20cm, 15cm), 
plot(layer(x=mod.agrid,y=adot[:,1,r_end],Geom.line,Theme(line_width=1.5mm)), 
layer(x=mod.agrid,y=adot[:,2,r_end],Geom.line, Theme(line_width=1.5mm,default_color=colorant"red")),
xintercept=[mod.amin], yintercept=[0], Geom.hline(color=colorant"black", size=0.5mm), 
Geom.vline(color=colorant"black", size=0.5mm),
Guide.title("Savings policy function"),
Guide.xlabel("Wealth, <i>a"),
Guide.ylabel("Savings, <i>s<sub>i</sub>(a)"), Coord.cartesian(xmin=mod.amin, xmax=mod.amax), Theme(major_label_font_size=6mm,
minor_label_font_size=4mm)) )

#### Stationary distributions

In the figure below we see the stationary distribution of agents across the two income states. Note that in the low income state (blue), there is a Dirac mass point on the borrowing constraint. That is, a large proportion of the low income agents are at the borrowing constraint in the stationary distribution. 

We can see that the high income state distribution is shifted right relative to the low income state distribution. This reflects the fact that high income state agents are saving while in that state.

In [8]:
draw(SVG(20cm, 15cm), 
plot(layer(x=mod.agrid,y=g_r[:,1,r_end],Geom.line,Theme(line_width=1.5mm)), layer(x=mod.agrid,y=g_r[:,2,r_end],Geom.line, Theme(line_width=1.5mm,default_color=colorant"red")),
xintercept=[mod.amin], Geom.vline(color=colorant"black", size=0.5mm), yintercept=[0], Geom.hline(color=colorant"black", size=0.5mm),
Guide.title("Stationary distributions"),
Guide.xlabel("Wealth, <i>a"),
Guide.ylabel("Density, <i>g<sub>i</sub>(a)"),
Coord.cartesian(xmin=mod.amin, xmax=1), Theme(major_label_font_size=6mm,
minor_label_font_size=4mm)) )